# Colab Specific initializations

In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [2]:
%cd /content/drive/MyDrive/tcr-embedding/example/

/content/drive/MyDrive/tcr-embedding/example


In [3]:
!pip install comet-ml scanpy scirpy

     |████████████████████████████████| 256kB 5.5MB/s 
     |████████████████████████████████| 10.3MB 7.1MB/s 
     |████████████████████████████████| 942kB 38.3MB/s 
     |████████████████████████████████| 512kB 37.4MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 122kB 41.9MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 3.1MB 42.2MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 14.1MB 248kB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 71kB 9.3MB/s 
  Created wheel for netifaces: filename=netifaces-0.10.9-cp37-cp37m-linux_x86_64.whl size=37412 sha256=8e91b0a73ca98985dfc744f2972aa3fe9514c641747332ac4b72f602d1222755
  Stored in directory: /root/.cache/pip/wheels/23/8f/f3/7054578f04c904f70757c5c85a6e2823baa69d42365526e93d
  Created 

In [4]:
# !pip install torch===1.7.1+cu110 torchvision===0.8.2+cu110 torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

# Real stuff

In [5]:
from comet_ml import Experiment
import scanpy as sc
import scirpy as ir
import pandas as pd
import torch
import yaml
import numpy as np
import matplotlib.pyplot as plt

In [6]:
import sys
sys.path.append('../')
import tcr_embedding as tcr

In [7]:
%load_ext autoreload
%autoreload 2

# Load dataset

### 10x Dataset

In [8]:
adata = sc.read_h5ad('../data/10x_CD8TC/highly_var_5000.h5ad')
adata

AnnData object with n_obs × n_vars = 178356 × 19707
    obs: 'IR_VJ_1_locus', 'IR_VJ_2_locus', 'IR_VDJ_1_locus', 'IR_VDJ_2_locus', 'IR_VJ_1_cdr3', 'IR_VJ_2_cdr3', 'IR_VDJ_1_cdr3', 'IR_VDJ_2_cdr3', 'IR_VJ_1_cdr3_nt', 'IR_VJ_2_cdr3_nt', 'IR_VDJ_1_cdr3_nt', 'IR_VDJ_2_cdr3_nt', 'IR_VJ_1_expr', 'IR_VJ_2_expr', 'IR_VDJ_1_expr', 'IR_VDJ_2_expr', 'IR_VJ_1_expr_raw', 'IR_VJ_2_expr_raw', 'IR_VDJ_1_expr_raw', 'IR_VDJ_2_expr_raw', 'IR_VJ_1_v_gene', 'IR_VJ_2_v_gene', 'IR_VDJ_1_v_gene', 'IR_VDJ_2_v_gene', 'IR_VJ_1_d_gene', 'IR_VJ_2_d_gene', 'IR_VDJ_1_d_gene', 'IR_VDJ_2_d_gene', 'IR_VJ_1_j_gene', 'IR_VJ_2_j_gene', 'IR_VDJ_1_j_gene', 'IR_VDJ_2_j_gene', 'IR_VJ_1_c_gene', 'IR_VJ_2_c_gene', 'IR_VDJ_1_c_gene', 'IR_VDJ_2_c_gene', 'IR_VJ_1_junction_ins', 'IR_VJ_2_junction_ins', 'IR_VDJ_1_junction_ins', 'IR_VDJ_2_junction_ins', 'has_ir', 'multi_chain', 'barcode', 'donor', 'cell_clono_cdr3_aa', 'cell_clono_cdr3_nt', 'CD3', 'CD19', 'CD45RA', 'CD4', 'CD8a', 'CD14', 'CD45RO', 'CD279_PD-1', 'IgG1', 'IgG2a', 'IgG2

In [9]:
adata = adata[:, adata.var['highly_variable']]
adata.shape

(178356, 5000)

In [10]:
(adata.X < 0)

<178356x5000 sparse matrix of type '<class 'numpy.bool_'>'
	with 0 stored elements in Compressed Sparse Row format>

In [11]:
pd.set_option('display.max_columns', None)
adata.obs

,IR_VJ_1_locus,IR_VJ_2_locus,IR_VDJ_1_locus,IR_VDJ_2_locus,IR_VJ_1_cdr3,IR_VJ_2_cdr3,IR_VDJ_1_cdr3,IR_VDJ_2_cdr3,IR_VJ_1_cdr3_nt,IR_VJ_2_cdr3_nt,IR_VDJ_1_cdr3_nt,IR_VDJ_2_cdr3_nt,IR_VJ_1_expr,IR_VJ_2_expr,IR_VDJ_1_expr,IR_VDJ_2_expr,IR_VJ_1_expr_raw,IR_VJ_2_expr_raw,IR_VDJ_1_expr_raw,IR_VDJ_2_expr_raw,IR_VJ_1_v_gene,IR_VJ_2_v_gene,IR_VDJ_1_v_gene,IR_VDJ_2_v_gene,IR_VJ_1_d_gene,IR_VJ_2_d_gene,IR_VDJ_1_d_gene,IR_VDJ_2_d_gene,IR_VJ_1_j_gene,IR_VJ_2_j_gene,IR_VDJ_1_j_gene,IR_VDJ_2_j_gene,IR_VJ_1_c_gene,IR_VJ_2_c_gene,IR_VDJ_1_c_gene,IR_VDJ_2_c_gene,IR_VJ_1_junction_ins,IR_VJ_2_junction_ins,IR_VDJ_1_junction_ins,IR_VDJ_2_junction_ins,has_ir,multi_chain,barcode,donor,cell_clono_cdr3_aa,cell_clono_cdr3_nt,CD3,CD19,CD45RA,CD4,CD8a,CD14,CD45RO,CD279_PD-1,IgG1,IgG2a,IgG2b,CD127,CD197_CCR7,HLA-DR,A0101_VTEHDTLLY_IE-1_CMV,A0201_KTWGQYWQV_gp100_Cancer,A0201_ELAGIGILTV_MART-1_Cancer,A0201_CLLWSFQTSA_Tyrosinase_Cancer,A0201_IMDQVPFSV_gp100_Cancer,A0201_SLLMWITQV_NY-ESO-1_Cancer,A0201_KVAELVHFL_MAGE-A3_Cancer,A0201_KVLEYVIKV_MAGE-A1_Cancer,A0201_CLLGTYTQDV_Kanamycin-B-dioxygenase,A0201_LLDFVRFMGV_EBNA-3B_EBV,A0201_LLMGTLGIVC_HPV-16E7_82-91,A0201_CLGGLLTMV_LMP-2A_EBV,A0201_YLLEMLWRL_LMP1_EBV,A0201_FLYALALLL_LMP2A_EBV,A0201_GILGFVFTL_Flu-MP_Influenza,A0201_GLCTLVAML_BMLF1_EBV,A0201_NLVPMVATV_pp65_CMV,A0201_ILKEPVHGV_RT_HIV,A0201_FLASKIGRLV_Ca2-indepen-Plip-A2,A2402_CYTWNQMNL_WT1-(235-243)236M_Y,A0201_RTLNAWVKV_Gag-protein_HIV,A0201_KLQCVDLHV_PSA146-154,A0201_LLFGYPVYV_HTLV-1,A0201_SLFNTVATL_Gag-protein_HIV,A0201_SLYNTVATLY_Gag-protein_HIV,A0201_SLFNTVATLY_Gag-protein_HIV,A0201_RMFPNAPYL_WT-1,A0201_YLNDHLEPWI_BCL-X_Cancer,A0201_MLDLQPETT_16E7_HPV,A0301_KLGGALQAK_IE-1_CMV,A0301_RLRAEAQVK_EMNA-3A_EBV,A0301_RIAAWMATY_BCL-2L1_Cancer,A1101_IVTDFSVIK_EBNA-3B_EBV,A1101_AVFDRKSDAK_EBNA-3B_EBV,B3501_IPSINVHHY_pp65_CMV,A2402_AYAQKIFKI_IE-1_CMV,A2402_QYDPVAALF_pp65_CMV,B0702_QPRAPIRPI_EBNA-6_EBV,B0702_TPRVTGGGAM_pp65_CMV,B0702_RPPIFIRRL_EBNA-3A_EBV,B0702_RPHERNGFTVL_pp65_CMV,B0801_RAKFKQLL_BZLF1_EBV,B0801_ELRRKMMYM_IE-1_CMV,B0801_FLRGRAYGL_EBNA-3A_EBV,A0101_SLEGGGLGY_NC,A0101_STEGGGLAY_NC,A0201_ALIAPVHAV_NC,A2402_AYSSAGASI_NC,B0702_GPAESAAGL_NC,NR(B0801)_AAKGRGAAL_NC,A0101_VTEHDTLLY_IE-1_CMV_binder,A0201_KTWGQYWQV_gp100_Cancer_binder,A0201_ELAGIGILTV_MART-1_Cancer_binder,A0201_CLLWSFQTSA_Tyrosinase_Cancer_binder,A0201_IMDQVPFSV_gp100_Cancer_binder,A0201_SLLMWITQV_NY-ESO-1_Cancer_binder,A0201_KVAELVHFL_MAGE-A3_Cancer_binder,A0201_KVLEYVIKV_MAGE-A1_Cancer_binder,A0201_CLLGTYTQDV_Kanamycin-B-dioxygenase_binder,A0201_LLDFVRFMGV_EBNA-3B_EBV_binder,A0201_LLMGTLGIVC_HPV-16E7_82-91_binder,A0201_CLGGLLTMV_LMP-2A_EBV_binder,A0201_YLLEMLWRL_LMP1_EBV_binder,A0201_FLYALALLL_LMP2A_EBV_binder,A0201_GILGFVFTL_Flu-MP_Influenza_binder,A0201_GLCTLVAML_BMLF1_EBV_binder,A0201_NLVPMVATV_pp65_CMV_binder,A0201_ILKEPVHGV_RT_HIV_binder,A0201_FLASKIGRLV_Ca2-indepen-Plip-A2_binder,A2402_CYTWNQMNL_WT1-(235-243)236M_Y_binder,A0201_RTLNAWVKV_Gag-protein_HIV_binder,A0201_KLQCVDLHV_PSA146-154_binder,A0201_LLFGYPVYV_HTLV-1_binder,A0201_SLFNTVATL_Gag-protein_HIV_binder,A0201_SLYNTVATLY_Gag-protein_HIV_binder,A0201_SLFNTVATLY_Gag-protein_HIV_binder,A0201_RMFPNAPYL_WT-1_binder,A0201_YLNDHLEPWI_BCL-X_Cancer_binder,A0201_MLDLQPETT_16E7_HPV_binder,A0301_KLGGALQAK_IE-1_CMV_binder,A0301_RLRAEAQVK_EMNA-3A_EBV_binder,A0301_RIAAWMATY_BCL-2L1_Cancer_binder,A1101_IVTDFSVIK_EBNA-3B_EBV_binder,A1101_AVFDRKSDAK_EBNA-3B_EBV_binder,B3501_IPSINVHHY_pp65_CMV_binder,A2402_AYAQKIFKI_IE-1_CMV_binder,A2402_QYDPVAALF_pp65_CMV_binder,B0702_QPRAPIRPI_EBNA-6_EBV_binder,B0702_TPRVTGGGAM_pp65_CMV_binder,B0702_RPPIFIRRL_EBNA-3A_EBV_binder,B0702_RPHERNGFTVL_pp65_CMV_binder,B0801_RAKFKQLL_BZLF1_EBV_binder,B0801_ELRRKMMYM_IE-1_CMV_binder,B0801_FLRGRAYGL_EBNA-3A_EBV_binder,A0101_SLEGGGLGY_NC_binder,A0101_STEGGGLAY_NC_binder,A0201_ALIAPVHAV_NC_binder,A2402_AYSSAGASI_NC_binder,B0702_GPAESAAGL_NC_binder,NR(B0801)_AAKGRGAAL_NC_binder,n_counts,log_counts,n_genes,mt_fraction,doublet_score,doublet,louvain
AAACGGGAGAAGATTC-1-donor_

## Preprocessing

### Clean multichain, and orphaned cells

In [12]:
print('Number of multichains: ', (adata.obs['multi_chain'] == 'True').sum())

Number of multichains:  147


In [13]:
adata[adata.obs['multi_chain'] == 'False']
adata.shape

(178356, 5000)

In [14]:
print('Number of NaNs in TRA: ', (adata.obs['IR_VJ_1_cdr3'] == 'nan').sum())
print('Number of None in TRA: ', (adata.obs['IR_VJ_1_cdr3'] == 'None').sum())

print('Number of NaNs in TRB: ', (adata.obs['IR_VDJ_1_cdr3'] == 'nan').sum())
print('Number of None in TRB: ', (adata.obs['IR_VDJ_1_cdr3'] == 'None').sum())

Number of NaNs in TRA:  25491
Number of None in TRA:  20098
Number of NaNs in TRB:  25491
Number of None in TRB:  4752


In [15]:
adata = adata[(adata.obs['IR_VJ_1_cdr3'] != 'None') & (adata.obs['IR_VJ_1_cdr3'] != 'nan') & 
              (adata.obs['IR_VDJ_1_cdr3'] != 'None') & (adata.obs['IR_VDJ_1_cdr3'] != 'nan')]
adata.shape

(128731, 5000)

In [16]:
print('Number of NaNs in TRA: ', (adata.obs['IR_VJ_1_cdr3'] == 'nan').sum())
print('Number of None in TRA: ', (adata.obs['IR_VJ_1_cdr3'] == 'None').sum())

print('Number of NaNs in TRB: ', (adata.obs['IR_VDJ_1_cdr3'] == 'nan').sum())
print('Number of None in TRB: ', (adata.obs['IR_VDJ_1_cdr3'] == 'None').sum())

Number of NaNs in TRA:  0
Number of None in TRA:  0
Number of NaNs in TRB:  0
Number of None in TRB:  0


### Represent aa chains as label sequence or one-hot encoding

Concatenate TRA and TRB sequence with '+' symbol in between

In [17]:
adata.obs['TRA+TRB'] = adata.obs['IR_VJ_1_cdr3'].astype(str) + '+' + adata.obs['IR_VDJ_1_cdr3'].astype(str)
adata.obs['TRA+TRB']

Trying to set attribute `.obs` of view, copying.


AAACGGGAGAAGATTC-1-donor_1       CAMREGSNDMRF+CASSIKGPFRTEAFF
AAACGGGTCGGACAAG-1-donor_1     CAREHMDSNYQLIW+CASSQLGRGDNEQFF
AAAGATGGTACAGACG-1-donor_1     CAALYNFNKFYF+CSARGLTADKDNYEQYF
AAAGTAGAGACGCTTT-1-donor_1    CAVIVWGNNRLAF+CASSLDRRLAGQETQYF
AAAGTAGAGCGCTTAT-1-donor_1       CAGDPDAGNNRKLIW+CAISEATDTQYF
                                           ...               
TTTGTCAGTACCCAAT-8-donor_4        CIVDTGANSKLTF+CASSESLDEKLFF
TTTGTCAGTCTAAACC-8-donor_4    CAMRGLEGGGNKLTF+CASSQDRGFGETQYF
TTTGTCAGTCTCTCGT-8-donor_4    CILRDGKGGGSQGNLIF+CASSSREPRELFF
TTTGTCATCCCACTTG-8-donor_4       CAVRDLMDTGRRALTF+CSATGQETQYF
TTTGTCATCTCTAAGG-8-donor_4     CGAIPTQGGSEKLVF+CATSGTENTGELFF
Name: TRA+TRB, Length: 128731, dtype: object

sanity check if data only contains the correct aa tokens

In [18]:
aa_tokens = adata.obs['TRA+TRB'].apply(lambda x: list(x))
unique_aa_tokens = sorted(set([x for sublist in aa_tokens for x in sublist]))
unique_aa_tokens

['+',
 'A',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'K',
 'L',
 'M',
 'N',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'V',
 'W',
 'Y']

Get maximum sequence length as pad value

In [19]:
pad = adata.obs['TRA+TRB'].str.len().max()
pad = int(pad)
pad

45

encode amino acids to unique id, '_' is pad character, '<' start and '>' end symbol '+' is the separation between TRA and TRB

In [20]:
aa_to_id = {'_': 0, 'A': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'K': 9, 'L': 10, 'M': 11, 'N': 12, 'P': 13,
            'Q': 14, 'R': 15, 'S': 16, 'T': 17, 'V': 18, 'W': 19, 'Y': 20, '+': 21, '<': 22, '>': 23}

In [21]:
# Returns aa_to_id, but we don't need it here. Only needed if the method should determine aa_to_id by itself
_ = tcr.utils.aa_encoding(adata, read_col='TRA+TRB', ohe_col='one_hot', label_col='tcr_seq', length_col='seq_len', pad=pad, aa_to_id=aa_to_id, start_end_symbol=True)
adata.obs[['TRA+TRB', 'tcr_seq', 'one_hot', 'seq_len']]

,TRA+TRB,tcr_seq,one_hot,seq_len
AAACGGGAGAAGATTC-1-donor_1,<CAMREGSNDMRF+CASSIKGPFRTEAFF>,"[22, 2, 1, 11, 15, 4, 6, 16, 12, 3, 11, 15, 5,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",30
AAACGGGTCGGACAAG-1-donor_1,<CAREHMDSNYQLIW+CASSQLGRGDNEQFF>,"[22, 2, 1, 15, 4, 7, 11, 3, 16, 12, 20, 14, 10...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",32
AAAGATGGTACAGACG-1-donor_1,<CAALYNFNKFYF+CSARGLTADKDNYEQYF>,"[22, 2, 1, 1, 10, 20, 12, 5, 12, 9, 5, 20, 5, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",32
AAAGTAGAGACGCTTT-1-donor_1,<CAVIVWGNNRLAF+CASSLDRRLAGQETQYF>,"[22, 2, 1, 18, 8, 18, 19, 6, 12, 12, 15, 10, 1...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",33
AAAGTAGAGCGCTTAT-1-donor_1,<CAGDPDAGNNRKLIW+CAISEATDTQYF>,"[22, 2, 1, 6, 3, 13, 3, 1, 6, 12, 12, 15, 9, 1...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",30
...,...,...,...,...
TTTGTCAGTACCCAAT-8-donor_4,<CIVDTGANSKLTF+CASSESLDEKLFF>,"[22, 2, 8, 18, 3, 17, 6, 1, 12, 16, 9, 10, 17,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",29
TTTGTCAGTCTAAACC-8-donor_4,<CAMRGLEGGGNKLTF+CASSQDRGFGETQYF>,"[22, 2, 1, 11, 15, 6, 10, 4, 6, 6, 6, 12, 9, 1...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",33
TTTGTCAGTCTCTCGT-8-donor_4,<CILRDGKGGGSQGNLIF+CASSSREPRELFF>,"[22, 2, 8, 10, 15, 3, 6, 9, 6, 6, 6, 16, 14, 6...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",33
TTTGTCATCCCACTTG-8-donor_4,<CAVRDLMDTGRRALTF+CSATGQETQYF>,"[22, 2, 1, 18, 15, 3, 10, 11, 3, 17, 6, 15, 15...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",30


### Antigen Binding Preprocessing

In [22]:
# antigen binding list
antigen_binding_list = ['A0101_VTEHDTLLY_IE-1_CMV_binder', 
                        'A0201_KTWGQYWQV_gp100_Cancer_binder', 
                        'A0201_ELAGIGILTV_MART-1_Cancer_binder', 
                        'A0201_CLLWSFQTSA_Tyrosinase_Cancer_binder', 
                        'A0201_IMDQVPFSV_gp100_Cancer_binder', 
                        'A0201_SLLMWITQV_NY-ESO-1_Cancer_binder', 
                        'A0201_KVAELVHFL_MAGE-A3_Cancer_binder', 
                        'A0201_KVLEYVIKV_MAGE-A1_Cancer_binder', 
                        'A0201_CLLGTYTQDV_Kanamycin-B-dioxygenase_binder', 
                        'A0201_LLDFVRFMGV_EBNA-3B_EBV_binder', 
                        'A0201_LLMGTLGIVC_HPV-16E7_82-91_binder', 
                        'A0201_CLGGLLTMV_LMP-2A_EBV_binder', 
                        'A0201_YLLEMLWRL_LMP1_EBV_binder', 
                        'A0201_FLYALALLL_LMP2A_EBV_binder', 
                        'A0201_GILGFVFTL_Flu-MP_Influenza_binder', 
                        'A0201_GLCTLVAML_BMLF1_EBV_binder', 
                        'A0201_NLVPMVATV_pp65_CMV_binder', 
                        'A0201_ILKEPVHGV_RT_HIV_binder', 
                        'A0201_FLASKIGRLV_Ca2-indepen-Plip-A2_binder', 
                        'A2402_CYTWNQMNL_WT1-(235-243)236M_Y_binder', 
                        'A0201_RTLNAWVKV_Gag-protein_HIV_binder', 
                        'A0201_KLQCVDLHV_PSA146-154_binder', 
                        'A0201_LLFGYPVYV_HTLV-1_binder', 
                        'A0201_SLFNTVATL_Gag-protein_HIV_binder', 
                        'A0201_SLYNTVATLY_Gag-protein_HIV_binder', 
                        'A0201_SLFNTVATLY_Gag-protein_HIV_binder', 
                        'A0201_RMFPNAPYL_WT-1_binder', 
                        'A0201_YLNDHLEPWI_BCL-X_Cancer_binder', 
                        'A0201_MLDLQPETT_16E7_HPV_binder', 
                        'A0301_KLGGALQAK_IE-1_CMV_binder', 
                        'A0301_RLRAEAQVK_EMNA-3A_EBV_binder', 
                        'A0301_RIAAWMATY_BCL-2L1_Cancer_binder',
                        'A1101_IVTDFSVIK_EBNA-3B_EBV_binder', 
                        'A1101_AVFDRKSDAK_EBNA-3B_EBV_binder', 
                        'B3501_IPSINVHHY_pp65_CMV_binder', 
                        'A2402_AYAQKIFKI_IE-1_CMV_binder',
                        'A2402_QYDPVAALF_pp65_CMV_binder',
                        'B0702_QPRAPIRPI_EBNA-6_EBV_binder',
                        'B0702_TPRVTGGGAM_pp65_CMV_binder',
                        'B0702_RPPIFIRRL_EBNA-3A_EBV_binder',
                        'B0702_RPHERNGFTVL_pp65_CMV_binder',
                        'B0801_RAKFKQLL_BZLF1_EBV_binder', 
                        'B0801_ELRRKMMYM_IE-1_CMV_binder', 
                        'B0801_FLRGRAYGL_EBNA-3A_EBV_binder',
                        'A0101_SLEGGGLGY_NC_binder', 
                        'A0101_STEGGGLAY_NC_binder',
                        'A0201_ALIAPVHAV_NC_binder', 
                        'A2402_AYSSAGASI_NC_binder',
                        'B0702_GPAESAAGL_NC_binder',
                        'NR(B0801)_AAKGRGAAL_NC_binder']

Create matrix, that determines antigen binding of each cell

In [23]:
binding_matrix = (adata.obs[antigen_binding_list] == 'True').values.astype(int)
binding_matrix.shape

(128731, 50)

Check if binding is limited to only one antigen
Result: Yes, every cell can only bind to one specific antigen, or binding is not determined

In [24]:
binding_matrix.sum(axis=1).shape  # just check the dimensions are correct and we have number of bindings for each cell

(128731,)

In [25]:
unique_value, count = np.unique(binding_matrix.sum(axis=1), return_counts=True)
for unique_value_, count_ in zip(unique_value, count):
    print(f'Cells with {unique_value_} bindings: {count_}')

Cells with 0 bindings: 66874
Cells with 1 bindings: 61857


Save antigen binding to column

In [26]:
label_to_binding = {label_id: antigen for label_id, antigen in enumerate(antigen_binding_list)}
label_to_binding[-1] = 'no_data'
binding_to_label = {antigen: label_id for label_id, antigen in enumerate(antigen_binding_list)}
binding_to_label['no_data'] = -1
binding_to_label == {v: k for k, v in label_to_binding.items()}  # sanity check, if both dicts are the reverse of each other

True

In [27]:
adata.obs['has_binding'] = binding_matrix.sum(axis=1).astype(bool)
adata.obs['has_binding']

AAACGGGAGAAGATTC-1-donor_1     True
AAACGGGTCGGACAAG-1-donor_1    False
AAAGATGGTACAGACG-1-donor_1    False
AAAGTAGAGACGCTTT-1-donor_1    False
AAAGTAGAGCGCTTAT-1-donor_1    False
                              ...  
TTTGTCAGTACCCAAT-8-donor_4    False
TTTGTCAGTCTAAACC-8-donor_4    False
TTTGTCAGTCTCTCGT-8-donor_4    False
TTTGTCATCCCACTTG-8-donor_4    False
TTTGTCATCTCTAAGG-8-donor_4    False
Name: has_binding, Length: 128731, dtype: bool

In [28]:
adata.obs['binding_label'] = np.argmax(binding_matrix, axis=1)
adata.obs['binding_label'][~adata.obs['has_binding']] = -1
adata.obs['binding_label'].head(15)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


AAACGGGAGAAGATTC-1-donor_1    29
AAACGGGTCGGACAAG-1-donor_1    -1
AAAGATGGTACAGACG-1-donor_1    -1
AAAGTAGAGACGCTTT-1-donor_1    -1
AAAGTAGAGCGCTTAT-1-donor_1    -1
AAAGTAGAGTTACGGG-1-donor_1    -1
AACCGCGGTCTGCGGT-1-donor_1    -1
AACGTTGTCGGATGTT-1-donor_1    -1
AACTCAGCAGGTTTCA-1-donor_1    -1
AACTGGTGTGCGAAAC-1-donor_1    -1
AACTTTCAGTAACCCT-1-donor_1    32
AACTTTCCATTACCTT-1-donor_1    -1
AAGGTTCCATTAGGCT-1-donor_1    -1
ACACCAACAGTCGATT-1-donor_1    -1
ACACCCTAGGAGTTTA-1-donor_1    29
Name: binding_label, dtype: int64

In [29]:
adata.obs['binding_name'] = adata.obs['binding_label'].map(label_to_binding)
adata.obs['binding_name'].head(15)

AAACGGGAGAAGATTC-1-donor_1       A0301_KLGGALQAK_IE-1_CMV_binder
AAACGGGTCGGACAAG-1-donor_1                               no_data
AAAGATGGTACAGACG-1-donor_1                               no_data
AAAGTAGAGACGCTTT-1-donor_1                               no_data
AAAGTAGAGCGCTTAT-1-donor_1                               no_data
AAAGTAGAGTTACGGG-1-donor_1                               no_data
AACCGCGGTCTGCGGT-1-donor_1                               no_data
AACGTTGTCGGATGTT-1-donor_1                               no_data
AACTCAGCAGGTTTCA-1-donor_1                               no_data
AACTGGTGTGCGAAAC-1-donor_1                               no_data
AACTTTCAGTAACCCT-1-donor_1    A1101_IVTDFSVIK_EBNA-3B_EBV_binder
AACTTTCCATTACCTT-1-donor_1                               no_data
AAGGTTCCATTAGGCT-1-donor_1                               no_data
ACACCAACAGTCGATT-1-donor_1                               no_data
ACACCCTAGGAGTTTA-1-donor_1       A0301_KLGGALQAK_IE-1_CMV_binder
Name: binding_name, dtype

In [30]:
# (adata.obs['has_binding'].astype(bool) == (adata.obs[antigen_binding_list] == 'True').sum(1).astype(bool)).all()

# Initialize and train model

In [31]:
config_name = 'bigru_paper_oriented'
with open(f'../config/{config_name}.yaml') as file:
     params = yaml.load(file)
experiment_name = '10x_random_split' + config_name
params

{'activation': 'leakyrelu',
 'batch_norm': True,
 'batch_size': 256,
 'dropout': 0.2,
 'hdim': 800,
 'loss_weights': [0.1, 1.0, 5e-05],
 'losses': ['MSE', 'CE'],
 'lr': 0.001,
 'scRNA_model_arch': 'MLP',
 'scRNA_model_hyperparams': {'activation': 'leakyrelu',
  'batch_norm': True,
  'dropout': 0.2,
  'gene_hidden': [800],
  'output_activation': 'relu'},
 'seq_model_arch': 'BiGRU',
 'seq_model_hyperparams': {'bidirectional': True,
  'dropout': 0.1,
  'embedding_dim': 64,
  'hidden_size': 256,
  'num_layers': 2},
 'shared_hidden': [200],
 'zdim': 100}

In [32]:
with open('../comet_ml_key/API_key.txt') as f:
    COMET_ML_KEY = f.read()

In [33]:
experiment = Experiment(api_key=COMET_ML_KEY, workspace='tcr', project_name='10x_GRU')

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/tcr/10x-gru/e64f46cf7b1a44fcaa87b1dac29baf0d



In [34]:
experiment.log_parameters(params)
experiment.log_parameter('experiment_name', experiment_name)

In [35]:
train_mask = np.zeros(len(adata))
train_mask[:int(len(adata)*0.8)] = 1
np.random.shuffle(train_mask)
train_mask = train_mask.astype(bool)
torch.save(train_mask, f'../saved_models/{experiment_name}_train_test_mask')

train_adata = adata[train_mask]
train_adata = train_adata[train_adata.obs['has_binding']]
test_adata = adata[~train_mask]
test_adata = test_adata[test_adata.obs['has_binding']]

In [36]:
model = tcr.models.joint_model.JointModel(
    adatas=[train_adata],  # adatas containing gene expression and TCR-seq
    names=['10x'],
    aa_to_id = aa_to_id,  # dict {aa_char: id}
    seq_model_arch=params['seq_model_arch'],  # seq model architecture
    seq_model_hyperparams=params['seq_model_hyperparams'],  # dict of seq model hyperparameters
    scRNA_model_arch=params['scRNA_model_arch'],
    scRNA_model_hyperparams=params['scRNA_model_hyperparams'],
    zdim=params['zdim'],  # zdim
    hdim=params['hdim'],  # hidden dimension of scRNA and seq encoders
    activation=params['activation'],  # activation function of autoencoder hidden layers
    dropout=params['dropout'],
    batch_norm=params['batch_norm'],
    shared_hidden=params['shared_hidden'],  # hidden layers of shared encoder / decoder
    gene_layers=[],  # [] or list of str for layer keys of each dataset
    seq_keys=[]  # [] or list of str for seq keys of each dataset
)

In [37]:
# print model architecture
model.model

JointModelTorch(
  (seq_encoder): BiGRUEncoder(
    (embedding): Embedding(24, 64, padding_idx=0)
    (gru): GRU(64, 256, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
    (output_layer): Linear(in_features=1024, out_features=800, bias=True)
  )
  (seq_decoder): BiGRUDecoder(
    (embedding): Embedding(24, 64, padding_idx=0)
    (hidden_state_layer): Linear(in_features=1600, out_features=512, bias=True)
    (gru): GRU(64, 256, num_layers=2, batch_first=True, dropout=0.1)
    (output_layer): Linear(in_features=256, out_features=24, bias=True)
  )
  (gene_encoder): MLP(
    (network): Sequential(
      (0): Sequential(
        (0): Linear(in_features=5000, out_features=800, bias=False)
        (1): BatchNorm1d(800, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.2, inplace=True)
        (3): Dropout(p=0.2, inplace=False)
      )
      (1): Sequential(
        (0): Linear(in_features=800, out_features=800, bias=Fal

In [ ]:
model.train(
    experiment_name=experiment_name,
    n_iters=None,
    n_epochs=300,
    batch_size=params['batch_size'],
    lr=params['lr'],
    losses=params['losses'],  # list of losses for each modality: losses[0] := scRNA, losses[1] := TCR
    loss_weights=params['loss_weights'],  # [] or list of floats storing weighting of loss in order [scRNA, TCR, KLD]
    val_split=0.25,
    metadata=[],
    validate_every=5,
    print_every=5,
    save_every=25,
    num_workers=0,
    verbose=1,  # 0: only tdqm progress bar, 1: val loss, 2: train and val loss
#     continue_training=True,
    device=None,
    comet=experiment
)

Create Dataloader
Dataloader created


Epoch:   0%|          | 0/300 [00:00<?, ?it/s]



Val Loss: 1.4726300239562988
Val scRNA Loss: 0.011768282391130924
Val TCR Loss: 1.4517579078674316
Val KLD Loss: 0.009103761054575443


Epoch:   2%|▏         | 5/300 [02:54<2:52:37, 35.11s/it]



Val Loss: 0.7095448970794678
Val scRNA Loss: 0.011488685384392738
Val TCR Loss: 0.6886442303657532
Val KLD Loss: 0.009411915205419064


...

Epoch:  83%|████████▎ | 250/300 [2:23:58<28:16, 33.94s/it]



Val Loss: 0.26489076018333435
Val scRNA Loss: 0.010725174099206924
Val TCR Loss: 0.2465043067932129
Val KLD Loss: 0.007661266252398491


Epoch:  85%|████████▌ | 255/300 [2:26:51<25:34, 34.09s/it]



Val Loss: 0.24799954891204834
Val scRNA Loss: 0.01072036474943161
Val TCR Loss: 0.22965316474437714
Val KLD Loss: 0.007626011967658997


Epoch:  87%|████████▋ | 260/300 [2:29:44<22:44, 34.10s/it]

In [ ]:
model.history

In [ ]:
model.train_history

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 10));
plt.subplot(221);
plt.plot(model.train_history['epoch'], model.train_history['loss'], '.-', label='Train loss');
plt.plot(model.history['epoch'], model.history['loss'], '.-', label='Val loss');
plt.legend();

plt.subplot(222);
plt.plot(model.train_history['epoch'], model.train_history['scRNA_loss'], '.-', label='Train scRNA loss');
plt.plot(model.history['epoch'], model.history['scRNA_loss'], '.-', label='Val scRNA loss');
plt.legend();

plt.subplot(223);
plt.plot(model.train_history['epoch'], model.train_history['TCR_loss'], '.-', label='Train TCR loss');
plt.plot(model.history['epoch'], model.history['TCR_loss'], '.-', label='Val TCR loss');
plt.xlabel('#Epochs');
plt.legend();

plt.subplot(224);
plt.plot(model.train_history['epoch'], model.train_history['KLD_loss'], '.-', label='Train KLD loss');
plt.plot(model.history['epoch'], model.history['KLD_loss'], '.-', label='Val KLD loss');
plt.xlabel('#');
plt.legend();


# UMAP Plot of latent space

### On Test Data

Filter cells with no binding data and only UMAP on high count antigen bindings

In [ ]:
test_adata = test_adata[test_adata.obs['has_binding']]

In [ ]:
test_adata.obs['binding_name'].value_counts()

In [ ]:
# only get antigen with high counts
antigen_count = test_adata.obs['binding_name'].value_counts()
high_antigen_count = antigen_count[antigen_count > len(test_adata)*0.01]
high_antigen_count

In [ ]:
test_adata

Use last saved model

In [ ]:
model.load(f'../saved_models/{experiment_name}_last_model.pt')

In [ ]:
z = model.get_latent(
    adatas=[test_adata],
    names=['10x'],
    batch_size=256,
    num_workers=0,
    gene_layers=[],
    seq_keys=[],
    metadata=antigen_binding_list + ['binding_name']
)

In [ ]:
z

In [ ]:
sc.pp.neighbors(z, use_rep='X')
sc.tl.umap(z)

In [ ]:
# plt.figure(figsize=(10,10))
for antigen in high_antigen_count.index:
  ax = sc.pl.umap(z, color=antigen, return_fig=True, alpha=0.3)
  experiment.log_figure(figure_name=f'test_{antigen}', figure=ax, step=model.epoch, overwrite=False)
  ax.clf()
ax = sc.pl.umap(z, color='binding_name', return_fig=True, alpha=0.2)
experiment.log_figure(figure_name=f'test_binding_name', figure=ax, step=model.epoch, overwrite=False)
ax.clf()

Use "best" saved model (currently based on val_loss)

In [ ]:
model.load(f'../saved_models/{experiment_name}_best_model.pt')

In [ ]:
z = model.get_latent(
    adatas=[test_adata],
    names=['10x'],
    batch_size=256,
    num_workers=0,
    gene_layers=[],
    seq_keys=[],
    metadata=antigen_binding_list + ['binding_name']
)

In [ ]:
sc.pp.neighbors(z, use_rep='X')
sc.tl.umap(z)

In [ ]:
# plt.figure(figsize=(10,10))
for antigen in high_antigen_count.index:
  ax = sc.pl.umap(z, color=antigen, return_fig=True, alpha=0.2)
  experiment.log_figure(figure_name=f'test_{antigen}', figure=ax, step=model.epoch, overwrite=False)
  ax.clf()
ax = sc.pl.umap(z, color='binding_name', return_fig=True, alpha=0.2)
experiment.log_figure(figure_name=f'test_binding_name', figure=ax, step=model.epoch, overwrite=False)
ax.clf()

### On Train Data

Filter cells with no binding data and only UMAP on high count antigen bindings

In [ ]:
train_adata = train_adata[train_adata.obs['has_binding']]

In [ ]:
train_adata.obs['binding_name'].value_counts()

In [ ]:
# only get antigen with high counts
antigen_count = train_adata.obs['binding_name'].value_counts()
high_antigen_count = antigen_count[antigen_count > len(train_adata)*0.01]
high_antigen_count

In [ ]:
train_adata

Use last saved model

In [ ]:
model.load(f'../saved_models/{experiment_name}_last_model.pt')

In [ ]:
z = model.get_latent(
    adatas=[train_adata],
    names=['10x'],
    batch_size=256,
    num_workers=0,
    gene_layers=[],
    seq_keys=[],
    metadata=antigen_binding_list + ['binding_name']
)

In [ ]:
z

In [ ]:
sc.pp.neighbors(z, use_rep='X')
sc.tl.umap(z)

In [ ]:
# plt.figure(figsize=(10,10))
for antigen in high_antigen_count.index:
  ax = sc.pl.umap(z, color=antigen, return_fig=True, alpha=0.3)
  experiment.log_figure(figure_name=f'train_{antigen}', figure=ax, step=model.epoch, overwrite=False)
  ax.clf()
ax = sc.pl.umap(z, color='binding_name', return_fig=True, alpha=0.2)
experiment.log_figure(figure_name=f'train_binding_name', figure=ax, step=model.epoch, overwrite=False)
ax.clf()

Use "best" saved model, (currently based on val loss)

In [ ]:
model.load(f'../saved_models/{experiment_name}_best_model.pt')

In [ ]:
z = model.get_latent(
    adatas=[train_adata],
    names=['10x'],
    batch_size=256,
    num_workers=0,
    gene_layers=[],
    seq_keys=[],
    metadata=antigen_binding_list + ['binding_name']
)

In [ ]:
sc.pp.neighbors(z, use_rep='X')
sc.tl.umap(z)

In [ ]:
# plt.figure(figsize=(10,10))
for antigen in high_antigen_count.index:
  ax = sc.pl.umap(z, color=antigen, return_fig=True, alpha=0.2)
  experiment.log_figure(figure_name=f'train_{antigen}', figure=ax, step=model.epoch, overwrite=False)
  ax.clf()
ax = sc.pl.umap(z, color='binding_name', return_fig=True, alpha=0.2)
experiment.log_figure(figure_name=f'train_binding_name', figure=ax, step=model.epoch, overwrite=False)
ax.clf()

# kNN prediction

In [ ]:
train_adata.obs['binding_name'].value_counts()

In [ ]:
test_adata.obs['binding_name'].value_counts()

In [ ]:
z_train = model.get_latent(
    adatas=[train_adata],
    names=['10x'],
    batch_size=256,
    num_workers=0,
    gene_layers=[],
    seq_keys=[],
    metadata=['binding_name', 'binding_label']
)

In [ ]:
z_test = model.get_latent(
    adatas=[test_adata],
    names=['10x'],
    batch_size=256,
    num_workers=0,
    gene_layers=[],
    seq_keys=[],
    metadata=['binding_name', 'binding_label']
)

In [ ]:
classes = 'binding_name'
model.kNN(z_train, z_test, classes, 5, 'distance')

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(z_test.obs[classes], z_test.obs['pred_'+classes]))
experiment.log_text(text=classification_report(z_test.obs[classes], z_test.obs['pred_'+classes]), step=model.epoch)

In [ ]:
experiment.log_table('confusion_matrix.csv', pd.crosstab(z_test.obs[classes], z_test.obs['pred_'+classes]))
pd.crosstab(z_test.obs[classes], z_test.obs['pred_'+classes])